# Datajoint_sandbox


In [1]:
import pandas as pd
import numpy as np
import os
import datajoint as dj
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
matplotlib.style.use('ggplot')
%matplotlib inline

<a id='connect'></a>
# Connect to the datajoint00 database
Username and password are same as Princeton login.

**Note**: You need to be on campus or using a VPN to connect to the database.

In [4]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.config['database.user'] = os.environ.get('DJ_DB_USER')
dj.config['database.password'] = os.environ.get('DJ_DB_PASS')
dj.conn()

Connecting ahoag@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) ahoag@datajoint00.pni.princeton.edu:3306

In [5]:
# db_lightsheet = dj.create_virtual_module('ahoag_lightsheet_demo','ahoag_lightsheet_demo')
# db_spockadmin = dj.create_virtual_module('ahoag_spockadmin_demo','ahoag_spockadmin_demo')
db_lightsheet = dj.create_virtual_module('a','u19lightserv_lightsheet')

## Sandbox begins

In [6]:
db_lightsheet.Request.ClearingBatch() & {'username':'hboele'}

username user in the lab,request_name,clearing_batch_number,clearing_protocol,antibody1,antibody2,antibody1_lot,antibody2_lot,clearing_progress,number_in_batch,perfusion_date,expected_handoff_date,clearer user in the lab,notes_for_clearer,link_to_clearing_spreadsheet
hboele,HJB_DCN_YFP,1,iDISCO+_immuno,"chicken antiGFP, Aves 1:500","Donkey antiChicken 647, 1:500",Aves GFP-1020 lot GFP879484,Jackson ImmunoResearch lot 136608,complete,2,2020-11-01,2020-11-16,ll3,We can try this in two animals and see how it looks like. (for sample: 20-MI-16684-01M) We can try this in two animals and see how it looks like (for sample: 20-MI-16684-02M),None
hboele,HJB_DCN_YFP,2,iDISCO+_immuno,"To be determined, maybe: chicken antiGFP, Aves 1:500","To be determined, maybe: Donkey antiChicken 647, 1:500",None,None,incomplete,6,2020-11-01,2020-11-16,ll3,We can start clearing for this brain and decide on antibodies later? (for sample: 20-MI-16685-01M) We can start clearing for this brain and decide on antibodies later? (for sample: 20-MI-16685-02M) We can start clearing for this brain and decide on antibodies later? (for sample: 20-MI-16686-01F) We can start clearing for this brain and decide on antibodies later? (for sample: 20-MI-16686-02F) We can start clearing for this brain and decide on antibodies later? (for sample: 20-MI-16687-01F) We can start clearing for this brain and decide on antibodies later? (for sample: 20-MI-16687-02F),None


In [7]:
db_lightsheet.Request.ClearingBatchSample() & {'username':'hboele'}

username user in the lab,request_name,sample_name,clearing_batch_number,clearing_protocol,antibody1,antibody2
hboele,HJB_DCN_YFP,20-MI-16684-01M,1,iDISCO+_immuno,"chicken antiGFP, Aves 1:500","Donkey antiChicken 647, 1:500"
hboele,HJB_DCN_YFP,20-MI-16684-02M,1,iDISCO+_immuno,"chicken antiGFP, Aves 1:500","Donkey antiChicken 647, 1:500"
hboele,HJB_DCN_YFP,20-MI-16685-01M,2,iDISCO+_immuno,"To be determined, maybe: chicken antiGFP, Aves 1:500","To be determined, maybe: Donkey antiChicken 647, 1:500"
hboele,HJB_DCN_YFP,20-MI-16685-02M,2,iDISCO+_immuno,"To be determined, maybe: chicken antiGFP, Aves 1:500","To be determined, maybe: Donkey antiChicken 647, 1:500"
hboele,HJB_DCN_YFP,20-MI-16686-01F,2,iDISCO+_immuno,"To be determined, maybe: chicken antiGFP, Aves 1:500","To be determined, maybe: Donkey antiChicken 647, 1:500"
hboele,HJB_DCN_YFP,20-MI-16686-02F,2,iDISCO+_immuno,"To be determined, maybe: chicken antiGFP, Aves 1:500","To be determined, maybe: Donkey antiChicken 647, 1:500"
hboele,HJB_DCN_YFP,20-MI-16687-01F,2,iDISCO+_immuno,"To be determined, maybe: chicken antiGFP, Aves 1:500","To be determined, maybe: Donkey antiChicken 647, 1:500"
hboele,HJB_DCN_YFP,20-MI-16687-02F,2,iDISCO+_immuno,"To be determined, maybe: chicken antiGFP, Aves 1:500","To be determined, maybe: Donkey antiChicken 647, 1:500"


In [9]:
db_lightsheet.Request.IdiscoPlusImmunoClearing()

AttributeError: type object 'Request' has no attribute 'IdiscoPlusImmunoClearing'

In [8]:
db_lightsheet.Request.ImagingBatchSample() & {'username':'hboele'}

username user in the lab,request_name,sample_name,clearing_batch_number,imaging_batch_number,imaging_request_number
hboele,HJB_DCN_YFP,20-MI-16684-01M,2,1,1
hboele,HJB_DCN_YFP,20-MI-16684-02M,2,1,1
hboele,HJB_DCN_YFP,20-MI-16685-01M,1,1,1
hboele,HJB_DCN_YFP,20-MI-16685-02M,1,1,1
hboele,HJB_DCN_YFP,20-MI-16686-01F,1,1,1
hboele,HJB_DCN_YFP,20-MI-16686-02F,1,1,1
hboele,HJB_DCN_YFP,20-MI-16687-01F,1,1,1
hboele,HJB_DCN_YFP,20-MI-16687-02F,1,1,1


In [ ]:
job_contents = db_spockadmin.ProcessingPipelineSpockJob()
unique_contents = dj.U('jobid_step0','username',).aggr(
    job_contents,timestamp='max(timestamp)')*job_contents

In [ ]:
unique_contents

In [ ]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = request_contents.ClearingBatch()
combined_contents = request_contents *clearing_batch_contents

In [ ]:
combined_contents

In [ ]:
username='soline'

In [ ]:
db_lightsheet.Request.ImagingChannel() & {'username':username}

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest() & {'username':username}

In [ ]:
db_lightsheet.Request.ImagingChannel() & {'username':username}

In [ ]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

In [ ]:
restrict_OR_list = [{"username":username},{"auditor":username}]
request_contents = request_contents & restrict_OR_list
imaging_request_contents = (request_contents*imaging_request_contents & restrict_OR_list).proj('imager',
                              'imaging_request_date_submitted',
                              'imaging_request_time_submitted',
                              'imaging_performed_date',
                              'imaging_progress')
processing_request_contents = (request_contents*processing_request_contents & restrict_OR_list).proj(
                            'processing_progress',
                            'processor')

In [ ]:
replicated_args = dict(is_archival='is_archival',number_of_samples='number_of_samples',description='description',
        species='species',datetime_submitted='datetime_submitted')

In [ ]:
sample_joined_contents = dj.U('username','request_name').aggr(
        request_contents * clearing_batch_contents,
        is_archival='is_archival',
        number_of_samples='number_of_samples',
        number_in_batch='number_in_batch',
        description='description',
        species='species',
        datetime_submitted='TIMESTAMP(date_submitted,time_submitted)',
        n_cleared='CONVERT(SUM(IF(clearing_progress="complete",number_in_batch,0)),char)').proj(
        **replicated_args,
            fraction_cleared='CONCAT(n_cleared,"/",CONVERT(number_of_samples,char))')

In [ ]:
imaging_joined_contents = sample_joined_contents.aggr(
    imaging_request_contents,
    **replicated_args,
    fraction_cleared='fraction_cleared',
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
    total_imaging_requests='CONVERT(COUNT(*),char)',
    keep_all_rows=True
    ).proj(**replicated_args,
        fraction_cleared='fraction_cleared',
        # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
        fraction_imaged='IF(n_imaged is NULL,"0/0",CONCAT(n_imaged,"/",total_imaging_requests))' 
        )

In [ ]:
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
    processing_request_contents,
    **replicated_args,
    fraction_cleared='fraction_cleared',
    fraction_imaged='fraction_imaged',
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)',
    keep_all_rows=True
    ).proj(
        **replicated_args,
        fraction_cleared='fraction_cleared',
        fraction_imaged='fraction_imaged',
        fraction_processed='IF(n_processed is NULL,"0/0",CONCAT(n_processed,"/",total_processing_requests))' 
        )

In [ ]:
processing_joined_contents

## Debugging all samples with auditor

In [ ]:
request_contents = db_lightsheet.Request()
request_contents = request_contents.proj('description','species','number_of_samples',
        'is_archival','auditor',
        datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

restrict_OR_list = [{"username":username},{"auditor":username}]
request_contents = request_contents & restrict_OR_list
sample_contents = (request_contents*sample_contents & restrict_OR_list).proj()

clearing_batch_contents = (request_contents*clearing_batch_contents & restrict_OR_list).proj(
    'clearer','clearing_progress','link_to_clearing_spreadsheet')
imaging_request_contents = (request_contents*imaging_request_contents & restrict_OR_list).proj(
    'imager','imaging_progress')
processing_request_contents = (request_contents*processing_request_contents & restrict_OR_list).proj(
    'processor','processing_progress')
sample_joined_contents = request_contents * sample_contents * clearing_batch_contents
replicated_args = dict(number_of_samples='number_of_samples',description='description',
        species='species',datetime_submitted='datetime_submitted',
        clearer='clearer',
        clearing_progress='clearing_progress',
        imager='imager',imaging_progress='imaging_progress',
        is_archival='is_archival',
        link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

In [ ]:
imaging_joined_contents = sample_joined_contents.aggr(
        imaging_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
        total_imaging_requests='COUNT(*)',
        keep_all_rows=True
        ).proj(**replicated_args,
               total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
               imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)'
            # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
            )

In [ ]:
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
        processing_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        processing_request_number='processing_request_number',
        processor='processor',
        processing_progress='processing_progress',
        total_imaging_requests='total_imaging_requests',
        n_processed='CONVERT(SUM(processing_progress="complete"),char)',
        total_processing_requests='CONVERT(COUNT(processing_progress),char)',
        keep_all_rows=True
        ).proj(
            **replicated_args,
            processing_request_number='IF(processing_request_number is NULL, "N/A",processing_request_number)',
            processor='processor',
            processing_progress='processing_progress',
            total_imaging_requests='total_imaging_requests',
            total_processing_requests='IF(n_processed is NULL,0,total_processing_requests)', 
            )

In [ ]:
processing_joined_contents

In [ ]:
contents = db_lightsheet.Request() & [{"username":username},{"auditor":username}]
contents

In [ ]:
imaging_batch_restrict_dict = dict(username=username,request_name=request_name,
    imaging_batch_number=imaging_batch_number,
    imaging_request_number=imaging_request_number)
sample_contents = db_lightsheet.Request.ImagingBatchSample() & imaging_batch_restrict_dict

In [ ]:
sample_contents

In [ ]:
restrict_dict = {'username':'lightserv-test','request_name':'test_new_imaging_request'}
db_lightsheet.Request() & restrict_dict

In [ ]:
db_lightsheet.Request.ImagingRequest & restrict_dict

In [ ]:
db_lightsheet.Request.ImagingBatch() & restrict_dict

In [ ]:
request_contents = db_lightsheet.Request()
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_batch_contents = db_lightsheet.Request.ImagingBatch()

imaging_request_contents = (clearing_batch_contents * sample_contents * \
    request_contents * imaging_batch_contents).\
    proj('clearer','clearing_progress',
    'imaging_request_date_submitted','imaging_request_time_submitted',
    'imaging_progress','imager','species','number_in_imaging_batch',
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)')

In [ ]:
imaging_request_contents = dj.U('username','request_name',
                                'imaging_batch_number',
                                'imaging_request_number').aggr(
    imaging_request_contents,clearer='clearer',
    clearing_progress='clearing_progress',
    datetime_submitted='datetime_submitted',
    imaging_progress='imaging_progress',imager='imager',
    species='species',number_in_imaging_batch='number_in_imaging_batch',
    all_samples_cleared='SUM(IF(clearing_progress="complete",1,0))=count(*)')

In [ ]:
imaging_request_contents & restrict_dict

In [ ]:
imaging_request_contents & restrict_dict

In [ ]:
contents_being_imaged = imaging_request_contents & 'all_samples_cleared=1' & \
    'imaging_progress="in progress"'

In [ ]:
contents_being_imaged & restrict_dict

In [ ]:
imaging_request_contents & 'imaging_request_number=2'

In [ ]:
sample_restrict_dict = \
    {'sample_name': 'test-001', 'imaging_request_number': '2', 'imaging_batch_number': '1'}

In [ ]:
db_lightsheet.Request.ClearingBatchSample()

In [ ]:
(db_lightsheet.Request() & 'username="pbibawi"').fetch('request_name')

In [ ]:
username='willmore'
request_name='201106_willmore'
sample_name = '201106_willmore-303'
imaging_request_number=1
processing_request_number=1
processing_restrict_dict = dict(username=username,request_name=request_name,
                sample_name=sample_name,imaging_request_number=imaging_request_number,
                processing_request_number=processing_request_number)

processing_request_contents = db_lightsheet.Request.ProcessingRequest() & processing_restrict_dict

processing_resolution_request_contents = db_lightsheet.Request.ProcessingResolutionRequest() & \
    processing_restrict_dict

processing_channel_contents = db_lightsheet.Request.ProcessingChannel() & processing_restrict_dict
image_resolution='1.1x'
processing_channel_contents_this_resolution = (processing_channel_contents & \
            f'image_resolution="{image_resolution}"')
restrict_dict = {'channel_name':'488','ventral_up':0}
this_processing_channel_content = (processing_channel_contents_this_resolution & \
                    restrict_dict)
this_processing_channel_content

In [ ]:
username='willmore'
request_name='201106_willmore'
imaging_batch_number=1
imaging_request_number=1

restrict_dict = dict(username=username,request_name=request_name,
                imaging_batch_number=imaging_batch_number)

channel_contents_all_samples = (db_lightsheet.Request.ImagingChannel() & f'request_name="{request_name}"' & \
    f'username="{username}"' )
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & \
    restrict_dict
imaging_resolution_contents = db_lightsheet.Request.ImagingResolutionRequest() & \
    restrict_dict
imaging_resolution_contents

In [ ]:
channel_contents_all_samples

In [ ]:
image_resolutions_no_processing = ["2x","3.6x","15x"]

In [ ]:
batch_unique_image_resolutions = set(imaging_resolution_contents.fetch('image_resolution'))
# batch_unique_image_re?solutions = set(['1.1x','3.6x'])

In [ ]:
have_processing_requests = any([x not in image_resolutions_no_processing for x in batch_unique_image_resolutions])
have_processing_requests

In [ ]:
set(imaging_request_contents.fetch('sample_name'))


In [ ]:
sample_names = set(imaging_resolution_contents.fetch('sample_name'))
sample_names

In [ ]:
data_rootpath = '/jukebox/LightSheetData/lightserv'

In [ ]:
message_body = ('Hello!\n\nThis is an automated email sent from lightserv, '
                        'the Light Sheet Microscopy portal at the Histology and Brain Registration Core Facility. '
                        'The raw data your request:\n'
                        f'request_name: "my-request"\n'
                        f'are now available on bucket here: \n\n')
for sample_name in sample_names:
    sample_basepath = os.path.join(data_rootpath,username,request_name,
    sample_name,f'imaging_request_number_{imaging_request_number}',  
    'rawdata')
    channel_contents_this_sample = channel_contents_all_samples & f'sample_name="{sample_name}"' 
    message_body += f'Sample name: {sample_name}:\n'
    for channel_dict in channel_contents_this_sample:
        channel_name = channel_dict['channel_name']
        image_resolution = channel_dict['image_resolution']
        ventral_up = channel_dict['ventral_up']
        rawdata_subfolder = channel_dict['rawdata_subfolder']
        if ventral_up:
            rawdata_fullpath = os.path.join(
                data_rootpath,username,request_name,sample_name,
                f'imaging_request_{imaging_request_number}',
                'rawdata',f'resolution_{image_resolution}_ventral_up',
                rawdata_subfolder)
            message_body += f'\tchannel {channel_name} (ventral up): {rawdata_fullpath}\n'
        else:
            rawdata_fullpath = os.path.join(
                data_rootpath,username,request_name,sample_name,
                f'imaging_request_{imaging_request_number}',
                'rawdata',f'resolution_{image_resolution}',
                rawdata_subfolder)
            message_body += f'\tchannel {channel_name}: {rawdata_fullpath}\n'
    message_body += '\n'
message_body += '\n'
if have_processing_requests:
    message_body += ('To start processing your data, '
            f'go to the processing management GUI: /some/path '
            'and find your sample to process.\n\n')
message_body += 'Thanks,\n\nThe Core Facility'
print(message_body)

In [ ]:
this_processing_channel_content_dict = this_processing_channel_content.fetch1()
this_processing_channel_content_dict

In [ ]:
processing_channel_contents_this_resolution

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest().fetch('atlas_name',limit=1)[0]

In [ ]:
processing_resolution_request_contents

In [ ]:
request_contents = db_lightsheet.Request()
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
imaging_channel_contents = db_lightsheet.Request.ImagingChannel()

In [ ]:
imaging_channel_contents & 'ventral_up=1'

In [ ]:
processing_resolution_contents = db_lightsheet.Request.ProcessingResolutionRequest() & {'username':'willmore'}
processing_resolution_contents 

In [ ]:
dj.Table._update(processing_resolution_contents,'atlas_name','allen_2017')

In [ ]:
clearing_batch_contents

In [ ]:
combined_contents = (clearing_batch_contents * request_contents).proj(
'description','number_in_batch','expected_handoff_date',
'clearing_protocol','species',
'clearer','clearing_progress','clearing_protocol','antibody1','antibody2',
datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
''' First get all entities that are currently being cleared '''

In [ ]:
idiscoplus_cleared_contents = combined_contents & 'clearing_protocol="iDISCO+_immuno"' & 'clearing_progress="complete"'
idiscoplus_cleared_contents

In [ ]:
df_idiscoplus_cleared = (idiscoplus_cleared_contents).fetch(
format='frame',order_by='datetime_submitted DESC')

In [ ]:
df_idiscoplus_cleared.to_csv("idiscoplus_already_cleared_asof_2020Nov12.csv")

In [ ]:
idisco_contents = db_lightsheet.Request.IdiscoPlusClearing()
idisco_contents

In [ ]:
idisco_website_cleared_contents = idisco_contents * idiscoplus_cleared_contents
idisco_website_cleared_contents

In [ ]:
idisco_results=idisco_website_cleared_contents.fetch(as_dict=True)
idisco_results

In [ ]:
df_idisco=pd.DataFrame(idisco_results)
df_idisco

In [ ]:
df_idisco.to_csv('idiscoplus_antibody_notes_already_cleared_asof_2020Nov12.csv')

In [ ]:
imaging_channel_contents & \
    {'username': 'ahoag',
     'request_name': 'test123',
     'sample_name': 'test123-001',
     'imaging_request_number': 1,
     'image_resolution': '1.3x',
     'channel_name': '488'}

In [ ]:
clearing_batch_contents.fetch(as_dict=True,limit=2)

In [ ]:
clearing_batch_contents.fetch?

In [ ]:
imaging_channel_contents & 'request_name="test_many_samples4"' 

In [ ]:
insert_dict = {'username': 'ahoag', 'request_name': 'test_many_samples4',
               'imaging_request_number': 1, 'image_resolution': '1.1x',
               'channel_name': '555',
               'injection_detection': 1,
               'sample_name': 'test_many_samples4-001'}


In [ ]:
imaging_merged_contents = (imaging_request_contents \
    * clearing_batch_contents * sample_contents * request_contents * imaging_channel_contents).\
    proj('clearer','clearing_progress',
    'imaging_request_date_submitted','imaging_request_time_submitted',
    'imaging_progress','imager','species',
    'image_resolution','channel_name',
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)') & \
    {'username':'ahoag',}

In [ ]:
replicated_args = dict(username='username',species='species',
                       datetime_submitted='datetime_submitted',
                       imager='imager',imaging_progress='imaging_progress')

In [ ]:
interm=dj.U("username","request_name","sample_name","imaging_request_number").aggr(
    imaging_merged_contents,new_col='GROUP_CONCAT(channel_name ORDER BY channel_name)',**replicated_args,
    image_resolution='image_resolution').\
    proj(comb_col='CONCAT(image_resolution,",",new_col)',**replicated_args,)
interm

In [ ]:
interm2=dj.U('request_name','sample_name','imaging_request_number').aggr(interm,
    **replicated_args,
    final_col='GROUP_CONCAT(comb_col)')
interm2

In [ ]:
dj.U('request_name','imaging_request_number','final_col').aggr(interm2,                               
    **replicated_args,
    number_in_batch='COUNT(*)')

In [ ]:
replicated_args = dict(username='username',
                       imager='imager',imaging_progress='imaging_progress')

In [ ]:
interm=dj.U("username","request_name","sample_name","imaging_request_number").aggr(
                        imaging_channel_contents,new_col='GROUP_CONCAT(channel_name ORDER BY channel_name)',
                        image_resolution='image_resolution').\
                        proj(comb_col='CONCAT(image_resolution,",",new_col)')
interm

In [ ]:
interm2=dj.U('request_name','sample_name','imaging_request_number').aggr(interm,
    sample_name='MIN(sample_name)',imaging_str='GROUP_CONCAT(comb_col)')
interm2

In [ ]:
interm3=dj.U('request_name','imaging_request_number','imaging_str').aggr(interm2,                               
    number_in_batch='COUNT(*)',).proj('request_name','number_in_batch')
interm3

In [ ]:
request_counter_dict = {}

In [ ]:
interm3.fetch(as_dict=True)

In [ ]:
imaging_channel_contents & 'request_name="test_many_samples3"'

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingResolutionRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents = db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents.fetch?

In [ ]:
db_lightsheet.Request.ImagingChannel().insert(channel_entry_dicts,skip_duplicates=True)

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
sorted(set(["a","b","a","a"]))

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ProcessingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents 

In [ ]:
var=4
s = f'{{var}} is the number'
s

In [ ]:
b = '1.3x_ventral_up'

In [ ]:
b if 'ventral_up' not in b else b.strip("_ventral_up")

In [ ]:
b.strip("_ventral_up")

In [ ]:
l=[{'id':'A','image_resolution':'4x','ventral_up':True},
   {'id':'B','image_resolution':'4x','ventral_up':False},
    {'id':'C','image_resolution':'1.3x','ventral_up':False},
    {'id':'D','image_resolution':'1.3x','ventral_up':False},
    {'id':'E','image_resolution':'1.3x','ventral_up':True}
  ]
tup_list = [(d['image_resolution'],d['ventral_up']) for d in l]
sorted(set(tup_list))

In [ ]:
all_channel_contents = imaging_channel_contents & 'request_name="test_dorsal_ventral"'

In [ ]:
results_list=all_channel_contents.fetch('image_resolution','ventral_up',as_dict=True)
tup_list = [(d['image_resolution'],d['ventral_up']) for d in results_list]
set(tup_list)
# all_channel_contents.fetch

In [ ]:
for a,b, in tup_list:
    print(a,b)

In [ ]:
bool(int('0'))

In [ ]:
job_contents = db_spockadmin.ProcessingPipelineSpockJob()
job_contents

In [ ]:
unique_contents = dj.U('jobid_step0','username',).aggr(
		job_contents,timestamp='max(timestamp)')*job_contents
unique_contents

In [ ]:
int(False)

In [ ]:
int(bool("False"))

In [ ]:
bool("False")

In [ ]:
s='Invitrogen rabbit polyclonal aldolase C'
len(s)

In [ ]:
b='Invitrogen donkey anti rabbit AlexaFluor 647'
len(b)

In [ ]:
c='1:50, 1:100, 1:200'
len(c)

In [ ]:
d={
    'date': datetime.date(2021, 1, 25),
    'brief_descriptor': 'aldolase C titration',
    'animal_model': 'mouse',
    'primary_antibody': 'Invitrogen rabbit polyclonal aldolase C',
    'primary_concentration': '1:50, 1:100, 1:200',
    'primary_order_info': 'Invitrogen PA512317, lot V13073112', 'secondary_antibody': 'Invitrogen donkey anti rabbit AlexaFluor 647', 'secondary_concentration': '1:200', 'secondary_order_info': 'Invitrogen A-31573', 'notes': 'waiting for titration decision'}

In [ ]:
date = '2020-02-01'
datetime.strptime(date,'%Y-%m-%d')

In [ ]:
[1,2]+[3]

In [ ]:
sample_imaging_dict = {}

## Debugging request_overview 

In [ ]:
username='ahoag'
request_name='test_imaging_batches_clearing_batches'
request_contents = db_lightsheet.Request() & f'request_name="{request_name}"' & \
            f'username="{username}"'
request_contents = request_contents.proj('description','species','number_of_samples',
    'is_archival',datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample() & f'request_name="{request_name}"' & f'username="{username}"' 
clearing_batch_contents = db_lightsheet.Request.ClearingBatch() & \
f'request_name="{request_name}"' & f'username="{username}"'
clearing_batch_sample_contents = db_lightsheet.Request.ClearingBatchSample() & \
f'request_name="{request_name}"' & f'username="{username}"' 
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & \
 f'request_name="{request_name}"' & f'username="{username}"' 
processing_request_contents = db_lightsheet.Request.ProcessingRequest() & \
 f'request_name="{request_name}"' & f'username="{username}"' 

In [ ]:
replicated_args = dict(number_of_samples='number_of_samples',description='description',
        species='species',clearing_progress='clearing_progress',
        clearer='clearer',
        imaging_progress='imaging_progress',imager='imager',
        is_archival='is_archival',
        link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

In [ ]:
sample_joined_contents = request_contents * clearing_batch_sample_contents * clearing_batch_contents.proj(
    'clearing_progress','clearer','link_to_clearing_spreadsheet')
imaging_joined_contents = sample_joined_contents.aggr(
        imaging_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
        total_imaging_requests='COUNT(*)',
        keep_all_rows=True
        ).proj(**replicated_args,
               total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
               imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)'
            # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
            )
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
        processing_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        processing_request_number='processing_request_number',
        processor='processor',
        processing_progress='processing_progress',
        total_imaging_requests='total_imaging_requests',
        n_processed='CONVERT(SUM(processing_progress="complete"),char)',
        total_processing_requests='CONVERT(COUNT(processing_progress),char)',
        keep_all_rows=True
        ).proj(
            **replicated_args,
            processing_request_number='IF(processing_request_number is NULL, "N/A",processing_request_number)',
            processor='processor',
            processing_progress='processing_progress',
            total_imaging_requests='total_imaging_requests',
            total_processing_requests='IF(n_processed is NULL,0,total_processing_requests)', 
            )

In [ ]:
sample_joined_contents

In [ ]:
imaging_joined_contents

In [ ]:
processing_joined_contents